In [42]:
#imports
import requests
import pandas as pd
import os
import datetime as dt
from google.cloud import bigquery
from google.cloud import storage
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim
import numpy as np
import meteostat as ms

In [43]:
CREDS = "C:/Users/tkkim/gcp_keys/capstone-team51-366963bafc54.json"
storage_client = storage.Client.from_service_account_json(json_credentials_path=CREDS,project='capstone-team51')
bq_client = bigquery.Client.from_service_account_json(json_credentials_path=CREDS,project='capstone-team51')
#client = bigquery.Client(project='capstone-team51')


In [44]:
bucket = storage_client.get_bucket('capstone-team51-data')

In [45]:
timestamp = dt.datetime.now()
timestamp = timestamp.strftime("%Y%m%d")

In [46]:
loglist = []

In [47]:
def getmaxdate(bq_client):
    QUERY = (
        'SELECT MAX(date) FROM `capstone-team51.chicago_data_prod.consolidated_table` '
        )
    query_job = bq_client.query(QUERY)  # API request
    df = query_job.to_dataframe()  # Waits for query to finish
    temp = df.iloc[0,0].to_pydatetime().date() + dt.timedelta(days=1)
    maxdate = temp.strftime("%Y-%m-%d")
    return maxdate


In [55]:
#primary data pull function

def get_data(name, db, maxrows, parameters, loglist, bucket, filepath):
    # some key info
    db_id = db
    limit = maxrows
    i = 0
    offset_counter = 0
    pickle = True
    data_name = name
    # dict for logging
    log_dict = {}

    # base url for the request
    baseurl = ("https://data.cityofchicago.org/resource/"
            f"{db_id}.json?")

    while pickle:
        #set our params
        # note that you have to URL econde 
        params = parameters + (
            f"&$limit={limit}"
            f"&$offset={offset_counter}"
            #f"&$$app_token={token}"
        )
        
        # make the request 
        data_req = requests.get(baseurl+params)

        # convert to json
        data_json = data_req.json()

        # make it a df
        df = pd.json_normalize(data_json)

        # write df
        
        
        # log
        minilog = {}
        minilog['name'] = name
        minilog['records'] = df.shape[0]
        minilog['chunk'] = i
        minilog['status'] = data_req.status_code
        minilog['offset'] = offset_counter
        minilog['date'] = timestamp
        
        
        if df.shape[0] == 0:
            break
        else:
            bucket.blob(f'{filepath}/{timestamp}_{data_name}_chunk_{i}.csv').upload_from_string(df.to_csv(index=False), 'text/csv')
            #df.to_json(f'{desti_folder}/{timestamp}_{data_name}_chunk_{i}.json',orient='records', lines=True)
            loglist.append(minilog)

        # increment the chunk count
        print(i)
        i += 1

        # increment the offset
        offset_counter += limit

        # check if we need to end the loop
        if df.shape[0] == 0:
            pickle = False
        elif data_req.status_code == 200:
            pickle = True
        else: 
            pickle = False
            print(data_req.status_code)
        return df

In [49]:
def get_weather_data(name, start_date, minilog, bucket, filepath):
    ## weather
    start = dt.datetime(int(start_date.split('-')[0]), int(start_date.split('-')[1]), int(start_date.split('-')[2]))
    endtime = dt.datetime.now().date().strftime("%Y-%m-%d")
    end = dt.datetime(int(endtime.split('-')[0]), int(endtime.split('-')[1]), int(endtime.split('-')[2]))
    # Create Point for chicago
    chicago = ms.Point(41.85, -87.65)

    dailyholder = ms.Daily(chicago, start, end)
    data_daily = dailyholder.fetch()
    data_daily[['tempmax_f', 'tempmin_f', 'tempavg_f']] = data_daily[['tmax', 'tmin', 'tavg']].apply(lambda x: (x * 9/5) + 32)
    bucket.blob(f'{filepath}/{timestamp}_weather_daily_data.csv').upload_from_string(data_daily.to_csv(), 'text/csv')
    #data_daily.to_csv(f'./data/{timestamp}_weather_daily_data.csv')

    minilog = {}
    minilog['name'] = name
    minilog['records'] = data_daily.shape[0]
    minilog['chunk'] = 0
    minilog['status'] = 200
    minilog['offset'] = 0
    minilog['date'] = timestamp
    #log_df = pd.DataFrame.from_dict(minilog,orient='index')
    loglist.append(minilog)

## Params and pulls

In [50]:
start_date = getmaxdate(bq_client)

In [56]:
# crime
crime_params = (
            f"$where=DATE%20%3E%3D%20%27{start_date}%27"
            f"&$order=DATE"
            #f"&$$app_token={token}"
            )

crimedata = get_data('crime', 'ijzp-q8t2', 500000, crime_params, loglist, bucket, 'raw_crimes')

0


In [70]:
end_date = pd.to_datetime(crimedata['date']).max().date()

In [78]:
end_date = dt.datetime.strftime(end_date, '%Y-%m-%d')

In [79]:
# dph env complaints
env_params = (
            f"$where=complaint_date%20BETWEEN%20%27{start_date}T00%3A00%3A00.000%27%20AND%20%27{end_date}T23%3A59%3A59.000%27"
            f"&$order=complaint_date"
            #f"&$$app_token={token}"
            )

get_data('dph_env', 'fypr-ksnz', 500000, env_params, loglist, bucket, 'raw_environmental')

0


,complaint_id,complaint_type,address,street_number,direction,street_name,street_type,inspector,complaint_date,inspection_log,data_source,modified_date,latitude,longitude,location.type,location.coordinates,complaint_detail
0,601485709,CONSTRUCTION AND DEMOLITION,1934 N SEMINARY AVE,1934,N,SEMINARY,AVE,379542,2024-01-02T00:00:00.000,[INSPECTION LOG #: 21281737 02-JAN-24 15:00:00...,DEPT. OF PUBLIC HEALTH,2024-01-08T00:00:00.000,41.917035884002566,-87.65552929240022,Point,"[-87.65552929240022, 41.917035884002566]",NaN
1,601485743,AIR POLLUTION WORK ORDER,6633 S MOZART ST,6633,S,MOZART,ST,249349,2024-01-02T00:00:00.000,[INSPECTION LOG #: 21262766 ] ON 1/2/2024 CDP...,DEPT. OF PUBLIC HEALTH,2024-01-03T00:00:00.000,41.772635074254794,-87.6943035754064,Point,"[-87.6943035754064, 41.772635074254794]",NaN
2,601485705,NOISE COMPLAINT,209 E OHIO ST,209,E,OHIO,ST,437489,2024-01-02T00:00:00.000,[INSPECTION LOG #: 21271208 02-JAN-24 12:25:00...,DEPT. OF PUBLIC HEALTH,2024-01-04T00:00:00.000,41.89245644738031,-87.62226362586607,Point,"[-87.62226362586607, 41.89245644738031]",NaN
3,601485714,CONSTRUCTION AND DEMOLITION,6458 S BISHOP ST,6458,S,BISHOP,ST,8386,2024-01-02T00:00:00.000,[INSPECTION LOG #: 21259084 02-JAN-24 14:10:00...,DEPT. OF PUBLIC HEALTH,2024-01-02T00:00:00.000,41.77597289860283,-87.66062216016986,Point,"[-87.66062216016986, 41.77597289860283]",NaN
4,601485703,CONSTRUCTION AND DEMOLITION,6 S LAFLIN ST,6,S,LAFLIN,ST,3308,2024-01-02T00:00:00.000,[INSPECTION LOG #: 21279463 02-JAN-24 11:15:00...,DEPT. OF PUBLIC HEALTH,2024-01-05T00:00:00.000,41.88132153435288,-87.66456137666425,Point,"[-87.66456137666425, 41.88132153435288]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,601494515,AIR POLLUTION WORK ORDER,2500 S PAULINA ST,2500,S,PAULINA,ST,129151,2024-01-26T00:00:00.000,[INSPECTION LOG #: 21370270 ] ON TODAYS DATE ...,DEPT. OF PUBLIC HEALTH,2024-01-26T00:00:00.000,41.84676817855723,-87.66850812645559,Point,"[-87.66850812645559, 41.84676817855723]",NaN
91,601495247,ILLEGAL DUMPING WORK ORDER,3238 N LSD BELMONT AV,3238,N,LSD BELMONT AV,NaN,417325,2024-01-29T00:00:00.000,[INSPECTION LOG #: 21383292 29-JAN-24 15:20:00...,DEPT. OF PUBLIC HEALTH,2024-01-29T00:00:00.000,NaN,NaN,NaN,NaN,NaN
92,601495204,ILLEGAL DUMPING WORK ORDER,2049 W DIVERSEY PKWY,2049,W,DIVERSEY,PKWY,417325,2024-01-29T00:00:00.000,[INSPECTION LOG #: 21383291 29-JAN-24 14:40:00...,DEPT. OF PUBLIC HEALTH,2024-01-29T00:00:00.000,41.93215369725963,-87.67992360839908,Point,"[-87.67992360839908, 41.93215369725963]",NaN
93,601495214,AIR POLLUTION WORK ORDER,4916 W HARRISON ST,4916,W,HARRISON,ST,10937,2024-01-29T00:00:00.000,[INSPECTION LOG #: 21382223 29-JAN-24 11:30:00...,DEPT. OF PUBLIC HEALTH,2024-01-29T00:00:00.000,41.87319225055247,-87.74759541072387,Point,"[-87.74759541072387, 41.87319225055247]",NaN


In [80]:
# 311 complaints
three11_params = (
            f"$where=(created_date%20BETWEEN%20%27{start_date}T00%3A00%3A00.000%27%20AND%20%27{end_date}T23%3A59%3A59.000%27)%20AND%20sr_short_code%20not%20in%20(%27311IOC%27)"
            f"&$order=created_date"
            )

get_data('311', 'v6vf-nfxy', 500000, three11_params, loglist, bucket, 'raw_311')



0


,sr_number,sr_type,sr_short_code,owner_department,status,origin,created_date,last_modified_date,closed_date,street_address,...,x_coordinate,y_coordinate,latitude,longitude,location.latitude,location.longitude,location.human_address,electricity_grid,parent_sr_number,created_department
0,SR24-00000001,Aircraft Noise Complaint,AVN,Aviation,Completed,Internet,2024-01-01T00:00:12.000,2024-01-01T00:30:39.000,2024-01-01T00:00:13.000,10510 W ZEMKE RD,...,1105376.767356118,1941043.788329337,41.994900001,-87.8877495,41.99490000094086,-87.8877495000019,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN
1,SR24-00000011,Dead Animal Pick-Up Request,SGQ,Streets and Sanitation,Completed,Mobile Device,2024-01-01T00:10:28.000,2024-01-03T12:43:07.000,2024-01-03T12:43:08.000,5918 N BROADWAY,...,1167215.940477632,1939524.284863489,41.989622099,-87.660317877,41.98962209909827,-87.66031787675992,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",M008,NaN,NaN
2,SR24-00000021,Aircraft Noise Complaint,AVN,Aviation,Completed,Internet,2024-01-01T00:18:59.000,2024-01-01T00:30:39.000,2024-01-01T00:19:00.000,10510 W ZEMKE RD,...,1105376.767356118,1941043.788329337,41.994900001,-87.8877495,41.99490000094086,-87.8877495000019,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN
3,SR24-00000024,Sign Repair Request - All Other Signs,PCE,CDOT - Department of Transportation,Open,Mobile Device,2024-01-01T00:19:46.000,2024-01-03T15:00:16.000,NaN,4427 N LAVERGNE AVE,...,1142322.286099968,1929094.046252221,41.961501001,-87.752142,41.961501000940714,-87.75214200000192,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",D011,NaN,NaN
4,SR24-00000034,Aircraft Noise Complaint,AVN,Aviation,Completed,Internet,2024-01-01T00:27:46.000,2024-01-01T00:30:44.000,2024-01-01T00:27:46.000,10510 W ZEMKE RD,...,1105376.767356118,1941043.788329337,41.994900001,-87.8877495,41.99490000094086,-87.8877495000019,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80591,SR24-00178956,Abandoned Vehicle Complaint,SKA,Streets and Sanitation,Open,Internet,2024-01-29T23:52:48.000,2024-01-29T23:52:53.000,NaN,2421 W HOLLYWOOD AVE,...,1159060.256301706,1937708.332174822,41.984811001,-87.690366,41.984811000940816,-87.69036600000192,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",J008,NaN,NaN
80592,SR24-00178958,Tobacco - General Complaint,BAG,BACP - Business Affairs and Consumer Protection,Open,Internet,2024-01-29T23:54:25.000,2024-01-30T00:01:17.000,NaN,6016 W NORTH AVE,...,1135973.161435688,1910153.913350678,41.909643001,-87.775938,41.909643000940555,-87.77593800000191,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",A018,SR24-00166846,NaN
80593,SR24-00178961,Buildings - Plumbing Violation,BBC,DOB - Buildings,Open,Phone Call,2024-01-29T23:56:22.000,2024-01-29T23:56:23.000,NaN,936 E 47TH ST,...,1183517.013043182,1874157.742298351,41.809887001,-87.602409,41.80988700094022,-87.6024090000019,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",S032,NaN,311 City Services
80594,SR24-00178962,Blue Recycling Cart,SRRC,Streets and Sanitation,Open,Mobile Device,2024-01-29T23:56:49.000,2024-01-29T23:56:51.000,NaN,2317 N SPAULDING AVE,...,1153834.721772935,1915311.279509728,41.923458001,-87.710184,41.92345800094063,-87.71018400000193,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",H017,NaN,NaN


In [81]:
# weather data
get_weather_data('weather', start_date, loglist, bucket, 'raw_weather')

In [82]:
log_df = pd.DataFrame(loglist)
bucket.blob(f'logs/{timestamp}_pull_log.csv').upload_from_string(log_df.to_csv(index=False), 'text/csv')